In [1]:
import os
import re
from datetime import date
import pandas as pd
from collections import defaultdict
import bisc.immport_templates as bs

#### JIRA BDD-2098  
**Goal:**  
    1. replace user def subjects by accessions in data matrix  
    2. figure out how to parse data matrix  

In [ ]:
##DIRECTORY CELL
beg = 'Please enter file path for '
subject_dir = input(beg + 'subject files: \n')
omrf_all = input(beg + "Please enter" omrf_all (all participatns omrf) filepath: \n")
omrf_all = input(beg + "ginger cohorot omrf filepath: \n")
planned_visits = input(beg + "planned visits filepath: \n")

#curration stuff
template_dir = input(beg + 'project templates: \n')
#subject_dir = 
workingfile = input(beg + 'all_participant OMRF deidentification: \n')
hai_data = input(beg + 'HAI_rawdata: \n')
inter_data = input(beg + 'interventions rawdata: \n')
assessments = input(beg+ 'assessments rawdata: \n')
wb_path = input(beg + ' ')
wb_data = input(beg + 'WB rawdata: \n')
elispot_data = input(beg + 'EliSPOT data: \n')             
elisa_data1 = input(beg + 'ELISA raw data: \n')
elisapath = input(beg + 'desired ')

In [2]:
#subject_dir = "/home/cthomas/Desktop/cris/projects/BDD2098/files/subjects/OMRF_subjects.txt"

subjects = {}
sub_sdy = {"1":"SDY196",
           "2":"SDY197",
           "3":"SDY198",
           "4":"SDY199",
           "5":"SDY200",
           "6":"SDY201",
}

with open(subject_dir, "r") as sbj:
    sbjfl = sbj.readline()
    for sbjlines in sbj:
        sbjlines = sbjlines.strip("\n")
        sbjstuff = sbjlines.split("\t")
        
        subjects[sbjstuff[3]] = sbjstuff[4]

print(len(set(subjects)))

203


In [24]:
#omrf_all = "/home/cthomas/Desktop/cris/projects/BDD2098/files/all_participants_OMRF.txt"
#omrf_ginger = "/home/cthomas/Desktop/cris/projects/BDD2098/files/gingers_cohort_OMRF.txt"

dfall = pd.read_table(omrf_all)
print(len(set(dfall.Barcode)))
dfg = pd.read_table(omrf_ginger)
print(len(set(dfg.Barcode)))

203
203


In [3]:
## planned visits
#planned_visits = "/home/cthomas/Desktop/cris/projects/BDD2098/files/planned_visits/OMRF_Planned_Visits.txt"
pv_firstday = {}    ## all the same visit days range

with open(planned_visits, "r") as pv:
    pvfl = pv.readline()
    for pvl in pv:
        pvstf = pvl.split("\t")
        pv_firstday[pvstf[1]] = pvstf[3]

In [28]:
files = [omrf_all, omrf_ginger]
day0 = defaultdict(dict)
for fl in files:
    ## Create new output file:
    tmp0 = fl.split(".")
    newfilename = "".join([tmp0[0], "_deidentified", tmp0[1]])
    with open(fl, "r") as omrf, open(newfilename, "w") as nwomrf:
        headers = omrf.readline()
        nwomrf.write("SDY_ACC\t" + headers)
        
        index_drawdates = []
        hdr = headers.split("\t")
        visit_count = defaultdict(dict)

        for hd in hdr:
            if hd.startswith("draw_"):
                index_drawdates.append(hdr.index(hd))

        for omrflines in omrf:
            omrfdata = omrflines.split("\t")
            tmp1 = subjects[omrfdata[0]]
            year = omrfdata[1]

            visit_count[omrfdata[0]][omrfdata[1]] = 0
            vc = 0

            for i in range(1, len(omrfdata)):
                data = omrfdata[i]
                data = data.strip()
                
                if i in index_drawdates:
                    visit_count[omrfdata[0]][omrfdata[1]] +=1
                    if re.match("\d\d\/\d\d\/\d\d\d\d", data):
                        if omrfdata[0] in day0:
                            if omrfdata[1] in day0[omrfdata[0]]:
                                mh0, dy0, yr0 = day0[omrfdata[0]][omrfdata[1]].split(r"/")
                                mh, dy, yr = data.split(r"/")
                                newdate = date(int(yr), int(mh), int(dy)) - date(int(yr0), int(mh0), int(dy0))
                                omrfdata[omrfdata.index(data)] = str(newdate.days + int(omrfdata[vc]))

                            else:
                                day0[omrfdata[0]][omrfdata[1]] = data
                                vc = int(omrfdata.index(data))
                                if visit_count[omrfdata[0]][omrfdata[1]] == 1:
                                    omrfdata[omrfdata.index(data)] = "0"
                                else:
                                    omrfdata[omrfdata.index(data)] = pv_firstday[str(visit_count[omrfdata[0]][omrfdata[1]])]
                        else:
                            day0[omrfdata[0]][omrfdata[1]] = data
                            vc = int(omrfdata.index(data))
                            if visit_count[omrfdata[0]][omrfdata[1]] == 1:
                                omrfdata[omrfdata.index(data)] = "0"
                            else:
                                omrfdata[omrfdata.index(data)] = pv_firstday[str(visit_count[omrfdata[0]][omrfdata[1]])]

            omrfdata[0] = tmp1
            omrfdata.insert(0, sub_sdy[year])            
            nwomrf.write("\t".join(omrfdata))
            
### FIX SUBSEQUENT INSTANCES OF SUBJECTS TO GET THE DAY 0
### FIX THE CASES WHERE DAY0 is not the first day - use planned visits maybe?

### CURATION STUFF

In [4]:
template_dir = "/home/cthomas/Desktop/cris/projects/BDD2098/files/templates"

In [5]:
subject_dir = "/home/cthomas/Desktop/cris/projects/BDD2098/files/subjects/OMRF_subjects.txt"

sub_acc = {}
sdy_yr = {
    "SDY196" : "1",
    "SDY197" : "2",
    "SDY198" : "3",
    "SDY199" : "4",
    "SDY200" : "5",
    "SDY201" : "6"
}

with open(subject_dir, "r") as sb:
    sbfl = sb.readline()
    for sblines in sb:
        sblines = sblines.strip("\n")
        sbstuff = sblines.split("\t")
        
        sub_acc[sbstuff[4]] = sbstuff[3]

planned_visits = "/home/cthomas/Desktop/cris/projects/BDD2098/files/planned_visits/OMRF_Planned_Visits.txt"
pv_acc = defaultdict(dict)

with open(planned_visits, "r") as pvi:
    pvifl = pvi.readline()
    for pviline in pvi:
        pviline = pviline.strip()
        pvistuff = pviline.split("\t")
        
        pv_acc[pvistuff[7]][pvistuff[1]] = pvistuff[0]   

empty = ""


In [6]:
pv_days = defaultdict(dict)
with open(workingfile) as alldata:
    pffle = alldata.readline()
    for lines in alldata:
        stuff = lines.split("\t")
        pv_days[stuff[0]][stuff[1]] = {
            pv_acc[stuff[0]]["1"] : stuff[9] ,
            pv_acc[stuff[0]]["2"] : stuff[19] ,
            pv_acc[stuff[0]]["3"] : stuff[29] ,
            pv_acc[stuff[0]]["4"] : stuff[40]
        }

In [60]:
### HAI ExpSample
hai_data = "/home/cthomas/Desktop/cris/projects/BDD2098/files/workingfiles/HAI_rawdata.txt"
## HAI
hai_t = bs.generatetemplate("hai", template_dir)

hai_prev = defaultdict(dict)
hai_prevrk = defaultdict(dict)
hai_pstv = defaultdict(dict)
hai_pstvrk = defaultdict(dict)

flustrains = {
    "SDY196" : "A/California/7/2004",
    "SDY197" : "A/Wisconsin/67/2005",
    "SDY198" : "A/Solomon Islands/3/2006",
    "SDY199" : "A/Uruguay/716/2007",
    "SDY200" : "A/Uruguay/716/2007",
    "SDY201" : "A/Perth/16/2009"
}

with open(hai_data, "r") as haid:
    haidfl = haid.readline()
    
    for haidline in haid:
        if not haidline.endswith("A"):
            haidline = haidline.strip()
            haiddata = haidline.split("\t")
            hai_prev[haiddata[0]][haiddata[1]] = haiddata[2]
            hai_prevrk[haiddata[0]][haiddata[1]] = haiddata[3]
            hai_pstv[haiddata[0]][haiddata[1]] = haiddata[4]
            hai_pstvrk[haiddata[0]][haiddata[1]] = haiddata[5]

with open(hai_t, "a") as hai:
    for studies in sdy_yr:
        extension = ".".join([studies, "txt"])
        addfile = "_".join(["omrf_ranks", extension])
        for subacc in hai_prev[studies]:
            prexpmtsplID = "_".join([sub_acc[subacc], sdy_yr[studies], studies, "HAI_prevac"])
            prebiosplID = "_".join([sub_acc[subacc], sdy_yr[studies], studies, "serum_prevac"])
            pstxpmtsplID = "_".join([sub_acc[subacc], sdy_yr[studies], studies, "HAI_postvac"])
            pstbiosplID = "_".join([sub_acc[subacc], sdy_yr[studies], studies, "serum_postvac"])
            prexpID = "_".join(["HAI_prevac", studies])
            pstxpID = "_".join(["HAI_postvac", studies])
            rk_prevax = " : ".join(["prevac_HAI_yr_rk", hai_prevrk[studies][subacc]]) 
            rk_postvax =" : ".join(["postvac_HAI_yr_rk", hai_pstvrk[studies][subacc]])
            prename = "_".join(["HAI_prevac", studies])
            postname = "_".join(["HAI_postvac", studies])
            hai.write("\t".join([empty, prexpmtsplID, prebiosplID, prexpID, empty, empty, empty, empty,
                                addfile, studies, "PTL7329;PTL6909", subacc, pv_acc[studies]["1"], "Serum",
                                empty, empty, empty, "0", "Days", "Time of initial vaccine administration", 
                                empty, prename, empty, "Neutralizing_Antibody_Titer", 
                                "Hemagglutination_Inhibition", empty, flustrains[studies], 
                                hai_prev[studies][subacc], rk_prevax]) + "\n") 
            hai.write("\t".join([empty, pstxpmtsplID, pstbiosplID, pstxpID, empty, empty, empty, empty,
                                addfile, studies, "PTL7329;PTL6909", subacc, "PlaceHolder_PV", "Serum",
                                empty, empty, empty, "PlaceHolderVisitDay", "Days", 
                                "Time of initial vaccine administration", empty, postname, empty,
                                "Neutralizing_Antibody_Titer", "Hemagglutination_Inhibition", empty,
                                flustrains[studies], hai_pstv[studies][subacc], rk_postvax]) + "\n") 
            
print("done")

This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/experimentSamples.HAI.txt was generated and can now be populated
done


In [21]:
## Interventions
inter_t = bs.generatetemplate("inter", template_dir)

inter_data = "/home/cthomas/Desktop/cris/projects/BDD2098/files/interventions_rawdata.txt"
counter_iv = {
    "SDY196" : 0,
    "SDY197" : 0,
    "SDY198" : 0,
    "SDY199" : 0,
    "SDY200" : 0,
    "SDY201" : 0
}
inters = defaultdict(dict)
with open(inter_data, "r") as it:
    itfl = it.readline()
    itfl = itfl.strip()
    headers = itfl.split("\t")
    
    for itlines in it:
        if re.search("yes", itlines):
            itlines = itlines.strip()
            itstuff = itlines.split("\t")
            inters[itstuff[0]][itstuff[1]] = []
            for i in range(len(itstuff)):
                if itstuff[i] == "yes":
                    inters[itstuff[0]][itstuff[1]].append(headers[i])

sub_med = defaultdict(dict)
pred = defaultdict(dict)
pred1 = "prednisone_10"
pred2 = "prednisone_10_30"
pred3 = "prednisone_30"

for truc in inters:
    for bid in inters[truc]:
        medocs = []
        for chou in inters[truc][bid]:
            if chou.startswith("Predn"):
                medocs.append("Prednisone")
                if pred1 in inters[truc][bid]:
                    if pred2 in inters[truc][bid]:
                        pred[truc][bid] = "less than 30mg"
                    else:
                        pred[truc][bid] = "less than 10mg"
                elif pred2 in inters[truc][bid]:
                    if pred3 in inters[truc][bid]:
                        pred[truc][bid] = "over 30mg or less"
                    else:
                        pred[truc][bid] = "between 10mg and 30mg"
                elif pred3 in inters[truc][bid]:
                    pred[truc][bid] = "over 30mg"
                else:
                    pred[truc][bid] = "UNK"
            elif not chou.startswith("pre"):
                medocs.append(chou)
        sub_med[truc][bid] = list(medocs)
        
                    
with open(inter_t, "a") as iv:
    for sdies in counter_iv:
        for sujets in sub_med[sdies]:
            for meds in sub_med[sdies][sujets]:
                counter_iv[sdies] += 1
                ivID = "_".join(["OMRF", sdies, str(counter_iv[sdies]).zfill(3)])
                dose = "UNK"
                if meds.startswith("Pred"):
                    dose = pred[sdies][sujets]
                ivname = meds
                ivcname = meds
                if meds.startswith("Az"):
                    ivname = "Imuran"
                    ivcname = "Azathioprine"
                if meds.startswith("Hydr"):
                    ivname = "Plaquenil"
                    ivcname = "Hydroxychloroquine"
                if meds.startswith("M"):
                    ivname = "Methotrexate"
                    ivcname = ivname
                iv.write("\t".join([empty, ivID, sujets, sdies, ivname, ivcname, "Concomitant Medication",
                                   dose, "0"]) + "\n")
                    

This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/interventions.txt was generated and can now be populated


In [38]:
## assessments
assessments_t1 = bs.generatetemplate("assmts", template_dir, "assessments.medH.SDY196.txt")
assessments_t2 = bs.generatetemplate("assmts", template_dir, "assessments.medH.SDY197.txt")
assessments_t3 = bs.generatetemplate("assmts", template_dir, "assessments.medH.SDY198.txt")
assessments_t4 = bs.generatetemplate("assmts", template_dir, "assessments.medH.SDY199.txt")
assessments_t5 = bs.generatetemplate("assmts", template_dir, "assessments.medH.SDY200.txt")
assessments_t6 = bs.generatetemplate("assmts", template_dir, "assessments.medH.SDY201.txt")
assessments_s1 = bs.generatetemplate("assmts", template_dir, "assessments.sle.SDY196.txt")
assessments_s2 = bs.generatetemplate("assmts", template_dir, "assessments.sle.SDY197.txt")
assessments_s3 = bs.generatetemplate("assmts", template_dir, "assessments.sle.SDY198.txt")
assessments_s4 = bs.generatetemplate("assmts", template_dir, "assessments.sle.SDY199.txt")
assessments_s5 = bs.generatetemplate("assmts", template_dir, "assessments.sle.SDY200.txt")
assessments_s6 = bs.generatetemplate("assmts", template_dir, "assessments.sle.SDY201.txt")

assmt_files = {
    "SDY196" : [assessments_t1, assessments_s1],
    "SDY197" : [assessments_t2, assessments_s2],
    "SDY198" : [assessments_t3, assessments_s3],
    "SDY199" : [assessments_t4, assessments_s4],
    "SDY200" : [assessments_t5, assessments_s5],
    "SDY201" : [assessments_t6, assessments_s6]
}
counter_as = {
    "SDY196" : 0,
    "SDY197" : 0,
    "SDY198" : 0,
    "SDY199" : 0,
    "SDY200" : 0,
    "SDY201" : 0
}

sub_age = {}
with open(subject_dir, "r") as sb:
    sbfl = sb.readline()
    for sblines in sb:
        sblines = sblines.strip("\n")
        sbstuff = sblines.split("\t")
        
        sub_age[sbstuff[4]] = sbstuff[6]
        
# pv_acc[study][order number]= planned_visit_acc
assess_medH = defaultdict(dict)
assess_sle = defaultdict(dict)
assess_dates = defaultdict(dict)
assess_names = []
assessments = "/home/cthomas/Desktop/cris/projects/BDD2098/files/assessments_rawdata.txt"
with open(assessments, "r") as ast:
    astfl = ast.readline()
    astfl = astfl.strip()
    asstnames = astfl.split("\t")
    assess_names = list(asstnames[5:33])
    
    for astlines in ast:
        astlines = astlines.strip()
        aststuff = astlines.split("\t")
        
        assess_medH[aststuff[0]][aststuff[1]] = list(aststuff[5:33])

        assess_sle[aststuff[0]][aststuff[1]] = {
            "PGA" : {"1": aststuff[33], "3": aststuff[34], "4": aststuff[35]},
            "PGA2" : {"1": aststuff[36], "3": aststuff[37], "4": aststuff[38]},
            "SLEDAI" : {"1": aststuff[39], "3": aststuff[40], "4": aststuff[41]},
            "SLAM" : {"1": aststuff[44], "3": aststuff[45], "4": aststuff[46]},
            "SLICC" : {"1": aststuff[47], "4": aststuff[48]},
            "Flare" : {"3": aststuff[42], "4": aststuff[43]}
        }

        assess_dates[aststuff[0]][aststuff[1]] = {"1" : "0", "3" : aststuff[3], "4" : aststuff[4]}
                
for study in assmt_files:
    with open(assmt_files[study][0], "a") as medH, open(assmt_files[study][1], "a") as sle:
        for subject in assess_medH[study]:
            medhist = []
            for j in range(0, len(assess_medH[study][subject])):
                counter_as[study] += 1
                assess_compID = "_".join(["OMRF", study, assess_names[j], str(counter_as[study]).zfill(4)])
                new_medh = "\t".join([assess_compID, pv_acc[study]["1"], assess_names[j], 
                                      assess_dates[study][subject]["1"], empty, sub_age[subject], "Years",
                                      empty, empty, empty, assess_medH[study][subject][j], empty, empty,
                                      empty, empty, empty, empty])
                medhist.append(new_medh)
            asst_panel = "_".join(["OMRF", study, "medical_history"])
            medH.write("\t".join([empty, subject, asst_panel, study, "Medical History", "Medical History",
                                  empty, empty, empty, empty, empty]))
            medH.write("\t".join(medhist) + "\n")
            
            slescores = []
            for scales in assess_sle[study][subject]:
                for visits in assess_sle[study][subject][scales]:
                    if not assess_sle[study][subject][scales][visits].startswith("N"):
                        counter_as[study] += 1
                        assess_comp = "_".join(["OMRF", study, scales, str(counter_as[study]).zfill(4)])
                        new_score = "\t".join([assess_comp, pv_acc[study][visits], scales, 
                                          assess_dates[study][subject][visits], empty, sub_age[subject], "Years",
                                          empty, empty, empty, assess_sle[study][subject][scales][visits], empty, 
                                          empty, empty, empty, empty, empty])
                        slescores.append(new_score)
            if slescores:
                slepanel = "_".join(["OMRF", study, "SLE_panel"])
                sle.write("\t".join([empty, subject, asst_panel, study, "SLE Panel", "Other",
                                      empty, empty, empty, empty, empty]))

                sle.write("\t".join(slescores) + "\n")
            

This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/assessments.medH.SDY196.txt was generated and can now be populated
This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/assessments.medH.SDY197.txt was generated and can now be populated
This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/assessments.medH.SDY198.txt was generated and can now be populated
This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/assessments.medH.SDY199.txt was generated and can now be populated
This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/assessments.medH.SDY200.txt was generated and can now be populated
This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/tem

In [47]:
wblots_t = bs.generatetemplate("other", template_dir)
wb_data = "/home/cthomas/Desktop/cris/projects/BDD2098/files/WB_rawdata.txt"

wb_path = "/home/cthomas/Desktop/cris/projects/BDD2098/files/wb"
wb_files = {
    "SDY196" : "omrf_WB_project4B_SDY196.txt",
    "SDY197" : "omrf_WB_project4B_SDY197.txt",
    "SDY198" : "omrf_WB_project4B_SDY198.txt",
    "SDY199" : "omrf_WB_project4B_SDY199.txt"
}

wbheaders = None
wbinfo = defaultdict(dict)
with open(wb_data, "r") as wbd:
    wbheaders = wbd.readline()
    
    for wbdline in wbd:
        wbdline = wbdline.strip()
        wbdstuff = wbdline.split("\t")
        if len(set(wbdstuff)) > 3:
            wbinfo[wbdstuff[0]][wbdstuff[1]] = wbdline

with open(wblots_t, "a") as wb:
    for sdy in wb_files:
        wbresultfile = os.path.join(wb_path, wb_files[sdy])
        with open(wbresultfile, "w") as wbr:
            wbr.write(wbheaders)

            for eachsub in wbinfo[sdy]:
                wbr.write(wbinfo[sdy][eachsub] + "\n")
                wbxpmtsplID = "_".join([sub_acc[eachsub], sdy_yr[sdy], sdy, "Project4B"])
                wbbiosplID = "_".join([sub_acc[eachsub], sdy_yr[sdy], sdy, "proteins"])
                wbxpID = "_".join(["WesternBlot", sdy])
                
                wb.write("\t".join([empty, wbxpmtsplID, wbbiosplID, wbxpID, empty, empty, wb_files[sdy],
                                   empty, empty, empty, sdy, "PTL5866", eachsub, empty, "Not_Specified",
                                   empty, empty, empty, empty, empty, empty, empty, wbxpID, empty, 
                                   "Protein_Quantification", "Western_Blot"]) + "\n")


This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/experimentSamples.Other.txt was generated and can now be populated


In [8]:
labtests_t = bs.generatetemplate("ltests", template_dir, "labTests.Project2.txt")
labtests_data = "/home/cthomas/Desktop/cris/projects/BDD2098/files/ELISA_rawdata_project2.txt"
sdtm = "/home/cthomas/Desktop/cris/projects/BDD2123/clinical.lk_lab_test_name.txt"

counter_lt = {
    "SDY196" : 0,
    "SDY197" : 0,
    "SDY198" : 0,
    "SDY199" : 0,
    "SDY200" : 0,
    "SDY201" : 0
}

sdtm_term = {}
with open(sdtm, "r") as sdtmt:
    for sdtmlines in sdtmt:
        sdtmstuff = sdtmlines.split("\t")
        sdtm_term[sdtmstuff[9]] = sdtmstuff[0]

labtestcodes = {
    "dsDNA_cri" : "C74913",
    "Ro_0_13" : "C92236",
    "La_0_18" : "C92237",
    "sm_nRNP_0_15" : "C120658",
    "sm_0_2" : "C92281",
    "Ribo_P_0_122" : "C120659",
    "ANA" : "C74916",
    "aPL_0_3" : "C102258",
    "total_IgG" : "C81971",
    "dsDNA_IU_117" : "C74913"
}
        
labtestresults = defaultdict(dict)
tmplabtestnames = []

with open(labtests_data, "r") as lbd:
    lbdfl = lbd.readline()
    lbdfl = lbdfl.strip()
    tmplabtestnames = lbdfl.split("\t")
    
    for lblines in lbd:
        lblines = lblines.strip()
        lbstuff = lblines.split("\t")

        labtestresults[lbstuff[0]][lbstuff[1]] = list(lbstuff)

labtestnames = []
for allnames in tmplabtestnames:
    tmpnm = allnames.split("_")
    nwnm = "_".join(tmpnm[1:])
    labtestnames.append(nwnm)
    
with open(labtests_t, "a") as lb:
    for etude in labtestresults:
        for sjt in labtestresults[etude]:
            pv_res = defaultdict(list)
            unit = "boolean"
            for k in range(3, 12):
                counter_lt[etude] += 1
                if not labtestresults[etude][sjt][k].endswith("A"):
                    if labtestnames[k].endswith("G"):
                        unit = "mg/ml"
                    lbtestname = "_".join([labtestnames[k], etude, str(counter_lt[etude]).zfill(4)])
                    lbres1 = "\t".join([lbtestname, sdtm_term[labtestcodes[labtestnames[k]]], 
                                      labtestresults[etude][sjt][k], unit])
                    pv_res["1"].append(lbres1)
            unit = "boolean"

            for l in range(13, 22):
                if not labtestresults[etude][sjt][l].endswith("A"):
                    counter_lt[etude] += 1
                    if labtestnames[l].endswith("G"):
                        unit = "mg/ml"
                    lbtestnm = "_".join([labtestnames[l], etude, str(counter_lt[etude]).zfill(4)])
                    lbres2 = "\t".join([lbtestnm, sdtm_term[labtestcodes[labtestnames[l]]], 
                                      labtestresults[etude][sjt][l], unit])
                    pv_res["2"].append(lbres2)
            unit = "boolean"
                
            for m in range(23, 33):
                if not labtestresults[etude][sjt][m].endswith("A"):
                    counter_lt[etude] += 1
                    if labtestnames[m].endswith("G"):
                        unit = "mg/ml"
                    if not labtestnames[m].endswith("type"):
                        ltestname = "_".join([labtestnames[m], etude, str(counter_lt[etude]).zfill(4)])
                        lbres3 = "\t".join([ltestname, sdtm_term[labtestcodes[labtestnames[m]]], 
                                          labtestresults[etude][sjt][m], unit])
                        pv_res["3"].append(lbres3)
            unit = "boolean"
    
            for n in range(13, 22):
                if not labtestresults[etude][sjt][m].endswith("A"):
                    counter_lt[etude] += 1
                    if labtestnames[n].endswith("G"):
                        unit = "mg/ml"
                    ltestnm = "_".join([labtestnames[n], etude, str(counter_lt[etude]).zfill(4)])
                    lbres4 = "\t".join([ltestnm, sdtm_term[labtestcodes[labtestnames[n]]], 
                                      labtestresults[etude][sjt][n], unit])
                    pv_res["4"].append(lbres4)
            
            drawdays = {
                "1" : labtestresults[etude][sjt][2], 
                "2" : labtestresults[etude][sjt][12],
                "3" : labtestresults[etude][sjt][22],
                "4" : labtestresults[etude][sjt][33]
            }
            for eachvisit in pv_res:
                biosampleID = "_".join([sub_acc[sjt], sdy_yr[etude], etude, "draw", eachvisit])
                labpanelID = "_".join([etude, "Project2", "draw", eachvisit])
                lbresult = "\t".join(pv_res[eachvisit])
                lb.write("\t".join([empty, biosampleID, labpanelID, etude, empty, sjt, pv_acc[etude][eachvisit],
                                   "Not_Specified", empty, empty, empty, drawdays[eachvisit], "Days",
                                   "Time of initial vaccine administration", empty, "Immunology Test",
                                   empty, lbresult]) + "\n")
    
    
    
    

 

This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/labTests.Project2.txt was generated and can now be populated


In [68]:
## EliSPOT
elispot_t = bs.generatetemplate("elispot", template_dir)
elispot_data = "/home/cthomas/Desktop/cris/projects/BDD2098/files/ELISPOT_rawdata.txt"

elispot_res = defaultdict(dict)

with open(elispot_data, "r") as elid:
    elidfl = elid.readline()
    
    for elilines in elid:
        elilines = elilines.strip()
        elistuff = elilines.split("\t")
        tmpeli = {
            "PHA" : {
                "prevac": {"spots" : elistuff[2], "rank" : elistuff[6] },
                "postvac" : {"spots" : elistuff[4] , "rank" : elistuff[8] }
            },
            "vacc" : {
                "prevac": {"spots" : elistuff[3] , "rank" : elistuff[7] },
                "postvac" : {"spots" : elistuff[5] , "rank" : elistuff[9] }
            }
        }
        elispot_res[elistuff[0]][elistuff[1]] = tmpeli
        
elispot_pv = {}

with open(elispot_t, "a") as eli:
    for stdy in elispot_res:
        elispot_pv[stdy] = {
            "prevac" : pv_acc[stdy]["1"],
            "postvac" : "PLACEHOLDER_POSTVAC_VISIT"
        }
        for subID in elispot_res[stdy]:
            pv_days[stdy][subID][elispot_pv[stdy]["postvac"]] = "PlaceHolderPVDay"
            for treatment in elispot_res[stdy][subID]:
                for time in elispot_res[stdy][subID][treatment]:
                    if not elispot_res[stdy][subID][treatment][time]["spots"].endswith("A"):
                        eliesID = "_".join([sub_acc[subID], sdy_yr[stdy], stdy, "ELISPOT", treatment, time ])
                        elibiospID = "_".join([sub_acc[subID], sdy_yr[stdy], stdy, "serum", treatment, time])
                        elixpID = "_".join(["ELISPOT", treatment, time, stdy])
                        treatmtID = "_".join([stdy, treatment, time])
                        elirk = "_".join(["ES", time, treatment, "spots_yr_rk"])
                        elirank = ":".join([elirk, elispot_res[stdy][subID][treatment][time]["rank"]])
                        eli.write("\t".join([empty, eliesID, elibiospID, elixpID, empty, treatmtID, empty,
                                            empty, empty, stdy, empty, subID, elispot_pv[stdy][time], 
                                            "Not_Specified", empty, empty, empty, 
                                             pv_days[stdy][subID][elispot_pv[stdy][time]], "Days",
                                            "Time of initial vaccine administration", empty, elixpID, empty,
                                            "Cellular_Activity", "ELISPOT", empty, "IFNG", 
                                            elispot_res[stdy][subID][treatment][time]["spots"], "100000",
                                             "SFUs", elirank]) + "\n")
 
                                       
                                       
                                       
                                       
                   

This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/experimentSamples.ELISPOT.txt was generated and can now be populated


In [75]:
project1_t = bs.generatetemplate("other", template_dir, "experimentSamples.Other.Project1.txt")
elisa_data1 = "/home/cthomas/Desktop/cris/projects/BDD2098/files/ELISA_rawdata_project1.txt"


elisa_path = "/home/cthomas/Desktop/cris/projects/BDD2098/files/project1"
elisa_files = {
    "SDY196" : "omrf_project1_SDY196.txt",
    "SDY197" : "omrf_project1_SDY197.txt",
    "SDY198" : "omrf_project1_SDY198.txt",
    "SDY199" : "omrf_project1_SDY199.txt",
    "SDY200" : "omrf_project1_SDY200.txt",
    "SDY201" : "omrf_project1_SDY201.txt"
}

airinfo = defaultdict(dict)
airheaders = ""
with open(elisa_data1, "r") as aird:
    airheaders = aird.readline()
    
    for airline in aird:
        airline = airline.strip()
        airstuff = airline.split("\t")
        if len(set(airstuff)) > 3:
            airinfo[airstuff[0]][airstuff[1]] = airline

with open(project1_t, "a") as pjt:
    for eachstudy in elisa_files:
        airresultfile = os.path.join(elisa_path, elisa_files[eachstudy])
        with open(airresultfile, "w") as air:
            air.write(airheaders)

            for eachsubject in airinfo[eachstudy]:
                air.write(airinfo[eachstudy][eachsubject] + "\n")
                airxpmtsplID = "_".join([sub_acc[eachsubject], sdy_yr[eachstudy], eachstudy, "Project1"])
                airbiosplID = "_".join([sub_acc[eachsubject], sdy_yr[eachstudy], eachstudy, "serum_Project1"])
                airxpID = "_".join(["Project1", eachstudy])
                airaddfile = "".join(["omrf_ranks_", eachstudy, ".txt"])
                pjt.write("\t".join([empty, airxpmtsplID, airbiosplID, airxpID, empty, empty, elisa_files[sdy],
                                   empty, empty, airaddfile, eachstudy, "PTL5866", eachsubject, empty, "Not_Specified",
                                   empty, empty, empty, empty, empty, empty, empty, airxpID, empty, 
                                   "Other", "ELISA"]) + "\n")

## created additional result files for the ranks -- omrf_ranks_SDY196.txt


This is a valid abbreviation to generate a template
/home/cthomas/Desktop/cris/projects/BDD2098/files/templates/experimentSamples.Other.Project1.txt was generated and can now be populated


In [ ]:
### Other test -- labtests
### ELISA
                                                                            